In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from src.data_loading import create_dataloader

In [2]:
# Define a simple regression model
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.fc(x)
depricated

In [3]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

In [4]:
# Set random seed for reproducibility
torch.manual_seed(42)
selected_features = ['time_to_tca', 'max_risk_estimate', 'c_time_lastob_start', 'c_time_lastob_end', 'c_ctdot_r']

# Hyperparameters
input_size = len(selected_features)  # Adjust based on your selected features
learning_rate = 0.001
num_epochs = 10

In [7]:
selected_columns = ['time_to_tca', 'max_risk_estimate', 'c_time_lastob_start', 'c_time_lastob_end', 'c_ctdot_r']
batch_size = 32

# Training DataLoader
train_dataloader = create_dataloader(
    csv_file='../../data/training/train_data.csv',
    selected_columns=selected_columns,
    is_train=True,
    batch_size=batch_size
)
print(f"Length of train_dataloader.dataset: {len(train_dataloader.dataset)}")
df = train_dataloader.dataset.data
df.head()



Columns in the dataset: Index(['time_to_tca', 'risk', 'max_risk_estimate', 'c_time_lastob_start',
       'c_time_lastob_end', 'c_ctdot_r'],
      dtype='object')
Length of features_train: 122714
Length of features_val: 30679
Length of train_dataloader.dataset: 122714


,time_to_tca,risk,max_risk_estimate,c_time_lastob_start,c_time_lastob_end,c_ctdot_r
0,1.566798,-10.204955,-7.834756,180.0,2.0,-0.732954
1,1.207494,-10.355758,-7.848937,180.0,2.0,-0.728759
2,0.952193,-10.345631,-7.847406,180.0,2.0,-0.729083
3,0.579669,-10.337809,-7.845880,180.0,2.0,-0.728092
4,0.257806,-10.391260,-7.852942,180.0,2.0,-0.727257


In [6]:
# Validation DataLoader
val_dataloader = create_dataloader(
    csv_file='../../data/training/train_data.csv',
    selected_columns=selected_columns,
    is_train=False,
    batch_size=batch_size
)
print(f"Length of val_dataloader.dataset: {len(val_dataloader.dataset)}")
df2 = val_dataloader.dataset.data
df2.head()


Columns in the dataset: Index(['time_to_tca', 'risk', 'max_risk_estimate', 'c_time_lastob_start',
       'c_time_lastob_end', 'c_ctdot_r'],
      dtype='object')
Length of features_train: 122714
Length of features_val: 30679
Length of val_dataloader.dataset: 30679


,time_to_tca,risk,max_risk_estimate,c_time_lastob_start,c_time_lastob_end,c_ctdot_r
0,1.566798,-10.204955,-7.834756,180.0,2.0,-0.732954
1,1.207494,-10.355758,-7.848937,180.0,2.0,-0.728759
2,0.952193,-10.345631,-7.847406,180.0,2.0,-0.729083
3,0.579669,-10.337809,-7.845880,180.0,2.0,-0.728092
4,0.257806,-10.391260,-7.852942,180.0,2.0,-0.727257


In [7]:
# Validation DataLoader
test_dataloader = create_dataloader(
    csv_file='../../data/testing/test_data.csv',
    selected_columns=selected_columns,
    is_train=False,
    batch_size=batch_size
)
print(f"Length of val_dataloader.dataset: {len(test_dataloader.dataset)}")
df3 = test_dataloader.dataset.data
df3.head()


Columns in the dataset: Index(['time_to_tca', 'risk', 'max_risk_estimate', 'c_time_lastob_start',
       'c_time_lastob_end', 'c_ctdot_r'],
      dtype='object')
Length of features_train: 18608
Length of features_val: 4653
Length of val_dataloader.dataset: 4653


,time_to_tca,risk,max_risk_estimate,c_time_lastob_start,c_time_lastob_end,c_ctdot_r
0,6.842095,-7.296967,-7.208941,180.0,2.0,-0.999539
1,6.571818,-7.282496,-7.199833,180.0,2.0,-0.999553
2,6.112986,-7.316053,-7.217886,180.0,2.0,-0.999520
3,5.921955,-7.334138,-7.228707,180.0,2.0,-0.999501
4,2.228761,-7.332267,-7.227312,180.0,2.0,-0.999495


In [8]:
# Create the model, criterion, and optimizer
model = RegressionModel(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train the model
train_model(model, train_dataloader, criterion, optimizer, num_epochs)

Epoch 1/10, Loss: 322.9403
Epoch 2/10, Loss: 217.2321
Epoch 3/10, Loss: 157.0487
Epoch 4/10, Loss: 125.3851
Epoch 5/10, Loss: 108.7981
Epoch 6/10, Loss: 98.4735
Epoch 7/10, Loss: 90.1566
Epoch 8/10, Loss: 82.7794
Epoch 9/10, Loss: 76.4602
Epoch 10/10, Loss: 71.3111


In [10]:
def evaluate_model(model, val_loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            total_loss += loss.item()
    avg_loss = total_loss / len(val_loader)
    return avg_loss


In [11]:
# Evaluate on the validation set
val_loss = evaluate_model(model, val_dataloader, criterion)
print(f'Validation Loss: {val_loss:.4f}')

Validation Loss: 68.7233


In [12]:
# Evaluate on the test set
test_loss = evaluate_model(model, train_dataloader, criterion)
print(f'Test Loss: {test_loss:.4f}')

Test Loss: 69.1371


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

df = load_data()
df_filtered = filter_data(df)

df_filtered.dropna(axis=0, how='any', inplace=True)

label_encode_column(df_filtered, "c_object_type")

features = df_filtered.drop(["risk"], axis=1)
target = df_filtered["risk"]

# Assuming 'selected_features' is a DataFrame and 'target' is a Series
selected_features = ['mahalanobis_distance', 'max_risk_estimate']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_filtered[selected_features], target, test_size=0.2, random_state=42)

# Train the model (Random Forest as an example)
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
